In [249]:
import torch
import torch.nn as nn

# force re-import
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [251]:
from Attractor import DirectionAttractorNet

B = 2
C = 4
mic_pos = [[0,0,0],[1,0,0],[0,1,0],[1,1,0]]

m = DirectionAttractorNet().to("cuda:0")

x  = torch.rand(B,C,16000).to("cuda:0")
angle = (torch.rand(B)*360).to("cuda:0")
y = m(x,angle,torch.tensor(mic_pos).unsqueeze(0).to("cuda:0"))
print(y.shape)

torch.Size([2, 4, 257, 126])
torch.Size([2, 4, 257, 126])


RuntimeError: expected scalar type Float but found ComplexFloat

In [63]:
x = torch.arange(0,3)
print(x.shape)

torch.Size([3])


# Shape

In [199]:
B = 2
C = 3
F = 4
T = 5

X = torch.rand(B,C,F,T,dtype=torch.cfloat)*2-1
M = torch.rand(B,F,T)
v = torch.rand(B,1,T)

print("== input == ")
print("X : {}".format(X.shape))
print("M : {}".format(M.shape))
print("v : {}".format(v.shape))

#print(M)
#print(v)

# X * X^H

X__ = torch.permute(X,(0,2,3,1))
X__ = torch.reshape(X__,(B*F*T,C,1))
M__ = torch.reshape(M,(B*F*T,1,1))
v__ = v.expand(B,F,T)
v__ = torch.reshape(v__,(B*F*T,1,1))
print("== batchwise == ")
print("X : {}".format(X__.shape))
print("M : {}".format(M__.shape))
print("v : {}".format(v__.shape))

#print(M__)
#print(v__)

R = torch.bmm(X__,torch.permute(X__,(0,2,1)))
print("R : {}".format(R.shape))
# masking
R *= v__
R *= M__

R = R + torch.eye(C)*1e-4

R_inv =  torch.inverse(R)

#R = torch.reshape(R,(B,F,T,C,C))

print(R.shape)
print(R_inv.shape)

W = torch.bmm(R,R_inv)
print(W.shape)

== input == 
X : torch.Size([2, 3, 4, 5])
M : torch.Size([2, 4, 5])
v : torch.Size([2, 1, 5])
== batchwise == 
X : torch.Size([40, 3, 1])
M : torch.Size([40, 1, 1])
v : torch.Size([40, 1, 1])
R : torch.Size([40, 3, 3])
torch.Size([40, 3, 3])
torch.Size([40, 3, 3])
torch.Size([40, 3, 3])


In [153]:
x = torch.rand(3,4,4)
x_inv = torch.inverse(x)
torch.bmm(x,x_inv)

tensor([[[ 1.0000e+00,  5.5879e-09, -5.9605e-08,  0.0000e+00],
         [ 0.0000e+00,  1.0000e+00,  5.9605e-08, -5.9605e-08],
         [ 5.9605e-08,  3.5390e-08,  1.0000e+00,  0.0000e+00],
         [-1.5646e-07, -5.0757e-08, -7.4506e-08,  1.0000e+00]],

        [[ 1.0000e+00,  1.7881e-07,  0.0000e+00,  7.1526e-07],
         [-1.4901e-07,  1.0000e+00,  4.7684e-07,  7.1526e-07],
         [-2.9802e-08,  5.9605e-08,  1.0000e+00,  1.1921e-07],
         [ 2.9802e-08, -1.4901e-08,  1.1921e-07,  1.0000e+00]],

        [[ 1.0000e+00,  7.1526e-07,  4.7684e-07, -9.5367e-07],
         [-1.9073e-06,  1.0000e+00,  0.0000e+00, -4.7684e-07],
         [-2.8610e-06,  0.0000e+00,  1.0000e+00,  4.7684e-07],
         [-2.3842e-06,  7.1526e-07,  0.0000e+00,  1.0000e+00]]])

In [173]:
C = torch.rand(3,1,dtype=torch.cfloat)
print(C)

torch.matmul(C,C.H)

tensor([[0.3625+0.4254j],
        [0.4639+0.6417j],
        [0.7367+0.2248j]])


tensor([[0.3124-9.7332e-10j, 0.4411-3.5302e-02j, 0.3627+2.3188e-01j],
        [0.4411+3.5302e-02j, 0.6270-1.1783e-08j, 0.4860+3.6847e-01j],
        [0.3627-2.3188e-01j, 0.4860-3.6847e-01j, 0.5932-6.2595e-09j]])

## Batch Beamforming

In [218]:
import torch

In [219]:
B = 2
C = 3
F = 4
T = 5

$$
\mathbf{w}(f) = \frac{\mathbf{R_u}^{-1}(f)\mathbf{h}(f)}{\mathbf{h}^H(f)\mathbf{R_u}^{-1}(f)\mathbf{h}(f)}
$$

In [238]:
X = torch.rand(B,C,F,T)
h = torch.rand(B,C,F,T)
R = torch.rand(B,C,C,F,T)

bh = torch.permute(h,(0,2,3,1))
bh = torch.reshape(bh,(B*F*T,C,1))
print(bh.shape)

bhT = torch.transpose(bh,1,2)
print(bhT.shape)

bR = torch.permute(R,(0,3,4,1,2))
bR = torch.reshape(bR,(B*F*T,C,C))
print(bR.shape)

bR_inv = torch.inverse(bR)
print(bR_inv.shape)

numer = torch.bmm(bR_inv,bh)
denom = torch.bmm(torch.bmm(bhT,bR_inv),bh)

print(numer.shape)
print(denom.shape)

bw = numer/denom
print(bw.shape)

bX = torch.permute(X,(0,2,3,1))
bX = torch.reshape(bX,(B*F*T,1,C))
print(bX.shape)

bY = torch.bmm(bX,bw)
print(bY.shape)

Y = torch.reshape(bY,(B,F,T))
print(Y.shape)

torch.Size([40, 3, 1])
torch.Size([40, 1, 3])
torch.Size([40, 3, 3])
torch.Size([40, 3, 3])
torch.Size([40, 3, 1])
torch.Size([40, 1, 1])
torch.Size([40, 3, 1])
torch.Size([40, 1, 3])
torch.Size([40, 1, 1])
torch.Size([2, 4, 5])


In [248]:
import torch.nn as nn
m = nn.ReLU
mm = nn.ReLU()
x = torch.rand(3,4)
print(x)
print(mm(x))

tensor([[0.0591, 0.1539, 0.9029, 0.2346],
        [0.5424, 0.7434, 0.9820, 0.9454],
        [0.4253, 0.4139, 0.0517, 0.0527]])
tensor([[0.0591, 0.1539, 0.9029, 0.2346],
        [0.5424, 0.7434, 0.9820, 0.9454],
        [0.4253, 0.4139, 0.0517, 0.0527]])


AttributeError: 'ReLU' object has no attribute 'ReLU'

tensor([[0.3922, 0.8384, 0.9527, 0.9151],
        [0.2924, 0.7863, 0.4554, 0.0034],
        [0.7606, 0.6014, 0.6268, 0.2857]])


NameError: name 'mm' is not defined

In [254]:
for i in range(-2) :
    print(i)

In [257]:
a = torch.rand(1,dtype=torch.cfloat)*10
b = torch.rand(1,dtype=torch.cfloat)*10

print(a)
print(b)
c = a.real*b.real + 1j*a.imag*b.imag
d = a*b
print(c)
print(d)

tensor([7.1207+6.0961j])
tensor([8.9406+3.6095j])
tensor([63.6628+22.0041j])
tensor([41.6587+80.2049j])
